In [1]:
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms

C:\Users\chand\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\chand\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
"""
# Instance-label split for Train and Test.
#
train_instances = torch.cat([instance for instance, label in trainloader])
train_labels = torch.cat([label for instance, label in trainloader])

test_instances = torch.cat([instance for instance, label in testloader])
test_labels = torch.cat([label for instance, label in testloader])
"""

'\n# Instance-label split for Train and Test.\n#\ntrain_instances = torch.cat([instance for instance, label in trainloader])\ntrain_labels = torch.cat([label for instance, label in trainloader])\n\ntest_instances = torch.cat([instance for instance, label in testloader])\ntest_labels = torch.cat([label for instance, label in testloader])\n'

# CNN Architecture Class

In [3]:
class CnnP2(nn.Module):
    """
    Convolutional Neural Network Architecture based on the description. 
    * Layers initialization.
    * Forward() function.
    """

    def __init__(self):
        super(CnnP2, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(2)
        self.linear = nn.Linear(64 * 3 * 3, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

# Original Training

In [19]:
def main(data):
    
    model = CnnP2()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(1):  # loop over the dataset multiple times

        correct = 0.0
        total = 0.0

        for i, data in enumerate(data, 0):
            
            inputs, labels = data
            optimizer.zero_grad()

            # forward + backward + optimize
            #
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(f'[Iter {epoch+1}] - Accuracy: {100 * correct // total} %')

    print('Finished Training!')

# MNIST data training

In [5]:
# Calling Train set.
#
train = torchvision.datasets.MNIST(root='MNIST/processed', train=True, download=True, transform=torchvision.transforms.ToTensor())

## Split 60K to 50K and 10K for Train and Val sets.
# Followed by applying loaders.
#
train_dl, valid_dl = torch.utils.data.random_split(train, [50000, 10000])
trainloader = torch.utils.data.DataLoader(train_dl, batch_size=16, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valid_dl, batch_size=16, shuffle=True, num_workers=2)

# Calling Test set.
#
test = torchvision.datasets.MNIST(root='MNIST/processed', train=False, download=True, transform=torchvision.transforms.ToTensor())
testloader = torch.utils.data.DataLoader(test, batch_size=16, shuffle=True, num_workers=2)

In [11]:
if __name__ == "__main__":
    main(trainloader)

[Iter 1] - Accuracy: 93.0 %
Finished Training!


# Data Augmentation

In [7]:
print(dir(transforms))

['AugMix', 'AutoAugment', 'AutoAugmentPolicy', 'CenterCrop', 'ColorJitter', 'Compose', 'ConvertImageDtype', 'ElasticTransform', 'FiveCrop', 'GaussianBlur', 'Grayscale', 'InterpolationMode', 'Lambda', 'LinearTransformation', 'Normalize', 'PILToTensor', 'Pad', 'RandAugment', 'RandomAdjustSharpness', 'RandomAffine', 'RandomApply', 'RandomAutocontrast', 'RandomChoice', 'RandomCrop', 'RandomEqualize', 'RandomErasing', 'RandomGrayscale', 'RandomHorizontalFlip', 'RandomInvert', 'RandomOrder', 'RandomPerspective', 'RandomPosterize', 'RandomResizedCrop', 'RandomRotation', 'RandomSolarize', 'RandomVerticalFlip', 'Resize', 'TenCrop', 'ToPILImage', 'ToTensor', 'TrivialAugmentWide', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_pil_constants', '_presets', 'autoaugment', 'functional', 'functional_pil', 'functional_tensor', 'transforms']


In [17]:
data_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=20),
    # transforms.RandomResizedCrop(size=128, scale=(0.8, 1.0)),
    transforms.ToTensor()
])

train_aug = torchvision.datasets.MNIST(root="MNIST/Augmented", train=True, download=True, transform=data_transform)
train_dl_aug, valid_dl_aug = torch.utils.data.random_split(train_aug, [50000, 10000])
trainloader_aug = torch.utils.data.DataLoader(train_dl_aug, batch_size=16, shuffle=True, num_workers=2)
valloader_aug = torch.utils.data.DataLoader(valid_dl_aug, batch_size=16, shuffle=True, num_workers=2)

In [20]:
if __name__ == "__main__":
    main(trainloader_aug)

[Iter 1] - Accuracy: 100.0 %


ValueError: too many values to unpack (expected 2)